In [3]:
![ -d QAOAKit ] || git clone https://github.com/melody0123/QAOAKit.git

import os
os.chdir("QAOAKit")
print(os.getcwd()) # debugging output

!pip install --upgrade pip
!pip install .

os.chdir("..")
print(os.getcwd()) # debugging output

/home/hyjing/Code/Quantum_Algorithm/QAOA_RET/QAOAKit
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1
Processing /home/hyjing/Code/Quantum_Algorithm/QAOA_RET/QAOAKit
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for QAOAKit: filename=qaoakit-0.1.13-py3-none-any.whl size=20203 sha256=7e2aadd2f014ce5034053ac0c8bfe4218625cc4cf7031dd1f39fd9fcc37a1a6a
  Stored in directory: /tmp/pip-ephem-wheel-cache-df9fn42v/wheels/49/6f/b4/5e964b05366ed5375b47e268534a7f1d4f1fc499519cab211d
Successfully built QAOAKit
  Attempting uninstall: QAOAKit
    Found existing installation: QAOAKit 0.1.13
    Uninstalling QAOAKit-0.1.13:
      Successfully uninstalled QAOAKit-0.1.13
/home/hyjing/Code/Quantum_Algorithm/QAOA_RET


In [4]:
!pip install --upgrade pip
!pip install pyzx
!pip install 'trasyn[qiskit,cupy-cuda12]'
!pip install matplotlib pylatexenc

In [5]:
!python -m QAOAKit.build_tables

Building tables (this may take a few minutes)...
Loading data, this may take a while
Building graph2angles table...
Building graph2pynauty table...
  0%|                                                     | 0/2 [00:00<?, ?it/s]
                                                                                
  0%|                                                     | 0/6 [00:00<?, ?it/s]
                                                                                
  0%|                                                    | 0/21 [00:00<?, ?it/s]
                                                                                
  0%|                                                   | 0/112 [00:00<?, ?it/s]
                                                                                
  0%|                                                   | 0/853 [00:00<?, ?it/s]
                                                                                
  0%|                                     

In [58]:
import networkx as nx
from qiskit import qpy
from qiskit_aer import AerSimulator
from QAOAKit import opt_angles_for_graph, angles_to_qaoa_format
from QAOAKit.qaoa import get_maxcut_qaoa_circuit

compute_on_the_fly = True
global qc
global new_qc
if compute_on_the_fly:
    # build a random 3-regular graph with 10 nodes
    graph = nx.random_regular_graph(d=3, n=10, seed=314)

    # grab optimal angles
    p = 3
    angles = angles_to_qaoa_format(opt_angles_for_graph(graph,p))

    # build circuit
    qc = get_maxcut_qaoa_circuit(graph, angles['beta'], angles['gamma'])

    # remove save_state operation
    new_qc = qc.copy_empty_like()
    for inst in qc:
        op = inst.operation
        if op.name != 'save_state':
            new_qc.append(op, inst.qubits, inst.clbits)
    
    # save circuit
    with open('algo_level_qc.qpy', 'wb') as qc_file:
        qpy.dump(qc, qc_file)
    with open('algo_level_qc_no_save_state.qpy', 'wb') as qc_file:
        qpy.dump(new_qc, qc_file)
else:
    # read circuit
    with open('algo_level_qc.qpy', 'rb') as qc_file:
        qc = qpy.load(qc_file)[0]
    with open('algo_level_qc_no_save_state.qpy', 'rb') as qc_file:
        new_qc = qpy.load(qc_file)[0]

# draw circuit
print(qc.count_ops())

OrderedDict([('cx', 90), ('u1', 75), ('u2', 70), ('save_state', 1)])


/home/hyjing/Code/Quantum_Algorithm/.venv/lib/python3.10/site-packages/QAOAKit/utils.py:218: UserWarning: Optimal angles not available, returning fixed angles for 3-regular graphs
  warnings.warn(


In [59]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import OptimizeCliffords, CommutativeCancellation, RemoveResetInZeroState, Unroll3qOrMore, BasisTranslator

# Step 1: Define safe passes
safe_passes = [
    RemoveResetInZeroState(),  # optional cleanup
    OptimizeCliffords(),       # optimize Clifford gates
    CommutativeCancellation(), # commute and cancel gates
]

# Step 2: Create a PassManager
pm = PassManager(safe_passes)

# Step 3: Run the passes manually
opt_qc = pm.run(new_qc)  # <-- NOT transpile()

# Step 4 (optional): Transpile to ensure basis gates only
from qiskit import transpile
opt_qc = transpile(opt_qc, optimization_level=1)

# Step 5: Done
print(dict(opt_qc.count_ops()))
# opt_qc.draw('mpl')



{'cx': 90, 'u1': 45, 'u3': 30, 'u2': 10}


In [61]:
import trasyn
from qiskit import qpy
from qiskit.quantum_info import Statevector, state_fidelity

# optimize t-gate count with transyn
global syned_qc
if compute_on_the_fly:
    # perform gate synthesis
    syned_qc = trasyn.synthesize_qiskit_circuit(circuit=opt_qc, u3_transpile=True, nonclifford_budget=50)
    
    # dump synthesized circuit to a file
    with open('trasyn_qc_reduced_t_gates.qpy', 'wb') as qc_file:
        qpy.dump(syned_qc, qc_file)
else:
    with open('trasyn_qc_reduced_t_gates.qpy', 'rb') as qc_file:
        syned_qc = qpy.load(qc_file)[0]

print("gate counts: ", dict(syned_qc.count_ops()))

# compute error after synthesis
original_state = Statevector(new_qc)
syned_state = Statevector(syned_qc)
fidelity = state_fidelity(state1=original_state, state2=syned_state)
print(f"state fidelity after synthesis: {fidelity}")

# draw cicuit
# syned_qc.draw(output='mpl')

RuntimeError: CuPy failed to load libnvrtc.so.12: OSError: libnvrtc.so.12: cannot open shared object file: No such file or directory

In [17]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import OptimizeCliffords, CommutativeCancellation, RemoveResetInZeroState, Unroll3qOrMore, BasisTranslator

basis = ['h', 'x', 'y', 'z', 's', 'cx', 't']
# Step 1: Define safe passes
safe_passes = [
    RemoveResetInZeroState(),  # optional cleanup
    OptimizeCliffords(),       # optimize Clifford gates
    CommutativeCancellation(), # commute and cancel gates
    Unroll3qOrMore(basis_gates=basis),
]

# Step 2: Create a PassManager
pm = PassManager(safe_passes)

# Step 3: Run the passes manually
opt_qc = pm.run(syned_qc)  # <-- NOT transpile()

# Step 4 (optional): Transpile to ensure basis gates only
from qiskit import transpile
opt_qc = transpile(opt_qc, basis_gates=basis, optimization_level=2)

# Step 5: Done
print(dict(opt_qc.count_ops()))
opt_qc.draw('mpl')


TranspilerError: 'Unable to translate the operations in the circuit: ["u", "cx"] to the backend\'s (or manually specified) target basis: {"h", "t", "reset", "y", "delay", "for_loop", "if_else", "store", "while_loop", "s", "measure", "barrier", "x", "snapshot", "z", "cx", "switch_case"}. This likely means the target basis is not universal or there are additional equivalence rules needed in the EquivalenceLibrary being used. For more details on this error see: https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.passes. BasisTranslator#translation-errors'